# Flaw Engine: Cloud Trainer (Gideon-gg101 Edition) 🚀

This notebook turns this Colab instance into a **Distributed Worker** for the Flaw Chess Engine.

### 🏁 Setup Options:
Run **ONLY ONE** of the setup options below.

## 1.1 Option A: Setup via GitHub (Recommended)
Use this if you have pushed your code to GitHub.

In [ ]:
GITHUB_URL = "https://github.com/Gideon-gg101/flaw-engine.git"

# Clean up previous attempts
!rm -rf /content/flaw

# Clone into /content/flaw
!git clone {GITHUB_URL} /content/flaw

# Verify
import os
if os.path.exists('/content/flaw'):
    print("✅ GitHub Sync Successful!")
    %cd /content/flaw
else:
    print("❌ FAILED to clone. Is your repository PUBLIC?")

## 1.2 Option B: Setup via Manual ZIP
Upload `flaw_cloud_bundle.zip` to the file explorer on the left, then run this.

In [ ]:
!rm -rf /content/flaw
!mkdir -p /content/flaw
!unzip -o /content/flaw_cloud_bundle.zip -d /content/flaw

import os
if os.path.exists('/content/flaw'):
    print("✅ Unzip Successful!")
    %cd /content/flaw
else:
    print("❌ FAILED to find ZIP. Did you upload 'flaw_cloud_bundle.zip'?")

## 2. Environment & Build
Run this to compile the engine.

In [ ]:
!apt-get install -y cmake g++
!pip install requests torch

%cd /content/flaw

!mkdir -p core/build
%cd core/build
!cmake ..
!make -j$(nproc)

# Move the built library to the root so Python can find it
!cp flaw_core*.so ../../

%cd /content/flaw
!ls -F

## 3. Start Training
Replace the `MASTER_URL` with your local `ngrok` URL.

In [ ]:
MASTER_URL = "https://scabbardless-haywood-spangly.ngrok-free.dev"

# Final verification and launch
!export PYTHONPATH=$PYTHONPATH:/content/flaw && cd /content/flaw && python3 -m ai.selfplay_worker --master {MASTER_URL}